### Example 1 - PandasAI & Feature Services

In this example you will explore the following concepts:

1. Connect to an Azure OpenAI Model (LLM), GPT-4o and run a simple test to make sure it's working correctly.
2. Use PandasAI Agent to create a sample dataframe powered by your LLM to ask it a few questions about the dataset, and validate the responses.
3. Taking the same approach from the previous step, connect a PandasAI Agent to an ArcGIS Online Feature Service, testing the questions and responses.

Let's begin!


Imports libraries for TOML file parsing, data manipulation with pandas, AI integration with Azure and pandas, and geographic information systems (GIS) functionalities.


In [ ]:
import toml
import pandas as pd
from langchain_openai import AzureChatOpenAI
from pandasai import Agent
from data.sample_dataframe import dataframe

from arcgis import GIS

Loads Azure configuration settings from a local TOML file and retrieves the first configuration set for use.


In [ ]:
azure_config = toml.load("config.toml")["configs"][0]
azure_config

Make the connection using the configuration options and test a simple chat message.


In [ ]:
llm = AzureChatOpenAI(
    openai_api_version=azure_config["api_version"],
    azure_deployment=azure_config["deployment_name"],
    api_key=azure_config["api_key"],
    azure_endpoint=azure_config["api_endpoint"],
    model=azure_config["model_name"],
    model_name=azure_config["model_name"],
    temperature=0,
)
response = llm.invoke("hi")
response.content

Create a simple Pandas dataframe using sample GDP and Happiness index data by country


In [ ]:
df = pd.DataFrame(dataframe)
df.head()

Create a PandasAI Agent using your LLM connection and your sample dataframe and begin asking questions.


In [ ]:
question1 = "Calculate the sum of the gdp of north american countries"
question2 = "What are the top five happiest countries in the world?"
question3 = "...?"

agent = Agent(df, config={"llm": llm, "enable_cache": False})

response = agent.chat(question1)

response

Connect to ArcGIS Online and load the wildfires feature layer


In [ ]:
gis = GIS()
wildfires_item_id = "d957997ccee7408287a963600a77f61f"
layer_id = 0
wildfires_item = gis.content.get(wildfires_item_id)
wildfires_feature_layer = wildfires_item.layers[layer_id]
wildfires_feature_layer

Query the wildfires layer for all the features and return the result as a Pandas dataframe


In [ ]:
wildfires_query_df = wildfires_feature_layer.query(
    where="1=1", out_fields="*", return_geometry=False, as_df=True
)
wildfires_query_df.head()

Create a PandasAI Agent using this dataframe and our LLM connection.

**Create your own question to ask and assign it to the final `question` variable that is commented out. Un-comment it, and run to see your results**


In [ ]:
wildfires_agent = Agent(wildfires_query_df, config={"llm": llm, "enable_cache": False})

question = "How many wildfires are there right now?"
# question = "List the fires that are 0 percent contained"
# question = "<insert question here>"

response = wildfires_agent.chat(question)
response